# Parsing chomsky normal form grammars (Dynamic Programming)

Chomsky normal form is a context free grammar with the rules in one of the following forms:
+ A → BC,   or
+ A → a,   or
+ S → ε,

This problem is a very nice case for dynamic programming. Historically bottom up (non-recursive) solutions are preferred as they provide a method to solve the problems by hand. On the othersize recursive solution can be much more elegant and readable. Compare the following function with the bottom-up version (https://en.wikipedia.org/wiki/CYK_algorithm)

## Optimal substructure
Here the optimal substructure means how a long solution can be obtained by combining solutions for subproblems. In this case, we have: A string w can be generated from S iff there is a way to split  w to xy such that S → XY is among the rules and X can generate x and y can generate Y. 

### Let’s define our variables first:
**R**: List of the rules. Every r in R is in this form : A → BC. We save A in r.lhs and BC in r.rhs
**w**: given string
**chart**: This is the dycamic programming table. Every entry of it has a list of pairs. For example  if 
`chart[‘abcd’]= [(1,2),(2,3)]` 
Then it mean you can generate  this string from rule number 1 if you split it at the 2nd position, or rule no 3 if you split it at position 3

# Implementing the parse

In [8]:
def parse(R,R_map,w,chart):
    if w in chart:
        return chart[w]
    if w in alphabet:
        for r in R_map[w]:
            chart[w].append((r,1))

    for (k,x,y) in split_string(w):
        parse(R,R_map, x, chart)
        parse(R,R_map, y, chart)
        for (r1_index,_),(r2_index,_) in zip(chart[x],chart[y]):
            (v1,_),(v2,_) = R[r1_index], R[r2_index]
            rhs = v1+v2
            for r in R_map[rhs]:
                chart[w].append((r,k))


# Testing The Parse

In [54]:
alphabet={'a','b','c','d'}
R=[('S','AB'),
('S','AA'),
('A','BC'),
('A','a'),
('A','b'),
('B','BA'),
('B','CA'),
('B','c'),
('B','b'),

('C','BB'),
('C','d')]


alphabet={'a','b','c','d'}
R=[('S','AB'),
('A','AA'),
('A','a'),
('B','BB'),
('B','b'),
  ]


#Create a dictionary (hashtable) with the right side of the rules as the key and the rule numbers as the values
from collections import defaultdict
R_map=defaultdict(list)
for (i,r) in enumerate(R):
    R_map[r[1]].append(i)
R_map   

defaultdict(list, {'AA': [1], 'AB': [0], 'BB': [3], 'a': [2], 'b': [4]})

# A simple helper function

In [10]:
def split_string(s):
    sl=[(k,s[:k],s[k:]) for k in range(1,len(s))]
    return sl
split_string('works_like_this')    

[(1, 'w', 'orks_like_this'),
 (2, 'wo', 'rks_like_this'),
 (3, 'wor', 'ks_like_this'),
 (4, 'work', 's_like_this'),
 (5, 'works', '_like_this'),
 (6, 'works_', 'like_this'),
 (7, 'works_l', 'ike_this'),
 (8, 'works_li', 'ke_this'),
 (9, 'works_lik', 'e_this'),
 (10, 'works_like', '_this'),
 (11, 'works_like_', 'this'),
 (12, 'works_like_t', 'his'),
 (13, 'works_like_th', 'is'),
 (14, 'works_like_thi', 's')]

## You need to install graphviz to run the following code

In [51]:
from Queue import Queue
from graphviz import Digraph
import graphviz

def generate(R,R_map,w,chart):
    
    q=Queue()
    dot = Digraph(engine='dot')
    
    dot.node('0', 'S')
    
    node_id=0
    q.put((w,'S',node_id))
    string=[]
    while not q.empty():
        w,U,Uid = q.get()
        found = False
        for (r_index,k) in chart[w]:
            if R[r_index][0]==U:
                found = True
                # U --> VW or U --> v
                V = R[r_index][1][0]
                node_id += 1
                dot.node(str(node_id), V)
                dot.edge(str(Uid), str(node_id))
                if len(R[r_index][1]) == 1:
                    break
                q.put((w[:k], V, node_id))
                W = R[r_index][1][1]
                node_id += 1
                dot.node(str(node_id), W)
                dot.edge(str(Uid), str(node_id))
                q.put((w[k:], W, node_id))
                break    
        if not found:
            print w," and ", U, "not found"
            return None
                    
    return dot


# Testing


In [57]:
#generate(R,'dadcbd')
sample_str = 'dadcbd'
sample_str = 'aabba'
chart=defaultdict(list)
parse(R,R_map,sample_str,chart)
dot=generate(R,R_map,sample_str,chart)
dot

aabba  and  S not found
